# Homework 5 - Design


## Theory

1. Using pencil and paper, compute the receptive field size at the input to the global average pooling layer for ResNet 50.

__Why use paper when you can use `Python`__

In [3]:
""" This calculation is for Resnet without bottle neck
Each block is 3x3 -> 3x3
"""

def add_filter(rf, f):
    return rf + f-1

def add_pool(rf, S):
    return (rf * S) - (S-1)

rf = 1
for i in range(3):
    rf = add_filter(rf,3)
    rf = add_filter(rf,3)
rf = add_pool(rf, 2)

for i in range(6):
    rf = add_filter(rf,3)
    rf = add_filter(rf,3)
rf = add_pool(rf, 2)

for i in range(4):
    rf = add_filter(rf,3)
    rf = add_filter(rf,3)
rf = add_pool(rf, 2)

for i in range(3):
    rf = add_filter(rf,3)
    rf = add_filter(rf,3)

#TAIL
rf = add_pool(rf, 2)
rf = add_pool(rf, 2)
rf = add_filter(rf,7)
print("ResNet Receptive Field (wOut bottle neck): {}".format(rf))

ResNet Receptive Field (wOut bottle neck): 951


In [4]:
""" This calculation is for Resnet with bottle neck
Each block is 1x1 -> 3x3 -> 1x1"""

rf = 1
for i in range(3):
    rf = add_filter(rf,3)
rf = add_pool(rf, 2)

for i in range(6):
    rf = add_filter(rf,3)
rf = add_pool(rf, 2)

for i in range(4):
    rf = add_filter(rf,3)
rf = add_pool(rf, 2)

for i in range(3):
    rf = add_filter(rf,3)

#TAIL
rf = add_pool(rf, 2)
rf = add_pool(rf, 2)
rf = add_filter(rf,7)
print("ResNet Receptive Field (w bottle neck): {}".format(rf))

ResNet Receptive Field (w bottle neck): 479


## Practice

In [ ]:
# data
DATA_NUM_CLASSES        = 200
DATA_CHANNELS           = 3
DATA_ROWS               = 64
DATA_COLS               = 64
#DATA_CROP_ROWS          = 28
#DATA_CROP_COLS          = 28

# model
MODEL_LEVEL_0_REPEATS   = 3
MODEL_LEVEL_1_REPEATS   = 3
MODEL_LEVEL_2_REPEATS   = 3

# training
TRAINING_BATCH_SIZE      = 32
TRAINING_SHUFFLE_BUFFER  = 5000
TRAINING_LR_MAX          = 0.001
# TRAINING_LR_SCALE        = 0.1
# TRAINING_LR_EPOCHS       = 2
TRAINING_LR_INIT_SCALE   = 0.01
TRAINING_LR_INIT_EPOCHS  = 5
TRAINING_LR_FINAL_SCALE  = 0.01
TRAINING_LR_FINAL_EPOCHS = 25

# training (derived)
TRAINING_NUM_EPOCHS = TRAINING_LR_INIT_EPOCHS + TRAINING_LR_FINAL_EPOCHS
TRAINING_LR_INIT    = TRAINING_LR_MAX*TRAINING_LR_INIT_SCALE
TRAINING_LR_FINAL   = TRAINING_LR_MAX*TRAINING_LR_FINAL_SCALE

DATA_PATH = 'F://Data/ImageNet/tiny-imagenet-200/'
SAVE_MODEL_PATH = pathlib.Path('F://Models/ImageNet_64/1/')
#!mkdir -p "$SAVE_MODEL_PATH"

AUTOTUNE = tf.data.experimental.AUTOTUNE

def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  return parts[-2] == CLASSES

def decode_image(img):
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.random_flip_left_right(img)
    #img = tf.image.random_crop(img, size=[DATA_CROP_ROWS, DATA_CROP_COLS, 3])
    return img

def process_path(path):
    """
    Input: file_path of a sample image
    Output: image in 3x64x64 float32 Tensor and one hot tensor
    """
    label = get_label(path)
    image = tf.io.read_file(path)
    image = decode_image(image)
    return image, label


def prepare_dataset(data_path, cache=False, shuffle_buffer_size=TRAINING_SHUFFLE_BUFFER):
    list_files = tf.data.Dataset.list_files(str(data_path/'*/*'))
    #map the above function to file_name dataset
    ds = list_files.map(process_path, num_parallel_calls=AUTOTUNE) 
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.repeat()
    ds = ds.batch(TRAINING_BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [ ]:
data_root = pathlib.Path(DATA_PATH)
CLASSES = os.listdir(data_root/'train')
train_ds_cachefile = prepare_dataset(data_root/'train', 
                                          cache=str(SAVE_MODEL_PATH.parent/"cache.tfcache"), 
                                          shuffle_buffer_size=TRAINING_SHUFFLE_BUFFER)

valid_ds_cachefile = prepare_dataset(data_root/'val', 
                                          cache=str(SAVE_MODEL_PATH.parent/"valid_cache.tfcache"), 
                                          shuffle_buffer_size=TRAINING_SHUFFLE_BUFFER)

test_ds_nocache = prepare_dataset(data_root/'test', 
                                          cache=False, 
                                          shuffle_buffer_size=TRAINING_SHUFFLE_BUFFER)

In [ ]:
class ResNet_Block(tf.keras.Model):
    def __init__(self, filters):
        super(ResNet_Block, self).__init__()
        
        self.conv1 = tf.keras.layers.Conv2D(filters, (1,1))
        self.bn1 = tf.keras.layers.BatchNormalization()
        
        self.conv2 = tf.keras.layers.Conv2D(filters,(3,3), padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()
        
        self.conv3 = tf.keras.layers.Conv2D(filters, (1,1))
        self.bn3 = tf.keras.layers.BatchNormalization()
    
    def call(self, x, training=True):
        f_x = self.bn1(self.conv1(x), training=training)
        f_x = tf.nn.relu(f_x)
        f_x = self.bn2(self.conv2(f_x), training=training)
        f_x = tf.nn.relu(f_x)
        f_x = self.bn3(self.conv3(f_x), training=training)
        f_x = tf.nn.relu(f_x)
        return x + f_x


def create_model(conv_block, level_repeats=[3,3,3], initial_filter_size=32):
    # encoder - input
    model_input = tf.keras.Input(shape=(DATA_ROWS, DATA_COLS, DATA_CHANNELS), name='input_image')
    x           = model_input
    
    x = tf.keras.layers.Conv2D(initial_filter_size, 
                               (7,7), 
                               strides=(1,1),
                               activation='relu',
                               padding='same')(x)
    
    
    filter_size = initial_filter_size
    for i in range(len(level_repeats)):
        x = tf.keras.layers.Conv2D(filter_size, (3,3), strides=(2,2), activation='relu', padding='same')(x)
        for n0 in range(level_repeats[i]):
            x = conv_block(filter_size)(x)    
        filter_size *= 2
        
        
    encoder_output = x
    # decoder
    y              = tf.keras.layers.GlobalAveragePooling2D()(encoder_output)
    decoder_output = tf.keras.layers.Dense(DATA_NUM_CLASSES, activation='softmax')(y)
    
    # forward path
    model = tf.keras.Model(inputs=model_input, outputs=decoder_output, name='Mymodel')
    # loss, backward path (implicit) and weight update
    model.compile(optimizer=tf.keras.optimizers.Adam(TRAINING_LR_MAX), loss='categorical_crossentropy', metrics=['acc'])
    # return model
    return model

In [ ]:
# create and compile model
model = create_model(ResNet_Block, level_repeats=[3,3,3], initial_filter_size=64)
# model description and figure
model.summary()

In [ ]:
# learning rate schedule
def lr_schedule(epoch):

    # staircase
    # lr = TRAINING_LR_MAX*math.pow(TRAINING_LR_SCALE, math.floor(epoch/TRAINING_LR_EPOCHS))

    # linear warmup followed by cosine decay
    if epoch < TRAINING_LR_INIT_EPOCHS:
        lr = (TRAINING_LR_MAX - TRAINING_LR_INIT)*(float(epoch)/TRAINING_LR_INIT_EPOCHS) + TRAINING_LR_INIT
    else:
        lr = (TRAINING_LR_MAX - TRAINING_LR_FINAL)*max(0.0, math.cos(((float(epoch) - TRAINING_LR_INIT_EPOCHS)/(TRAINING_LR_FINAL_EPOCHS - 1.0))*(math.pi/2.0))) + TRAINING_LR_FINAL
    return lr

plt.plot(range(1,TRAINING_NUM_EPOCHS+1), [lr_schedule(i) for i in np.arange(TRAINING_NUM_EPOCHS)+1])
plt.title("Learning Rate Schedule")
plt.xlabel("Epoch")
plt.ylabel("l.r.")
plt.plot()

In [ ]:
# plot training accuracy and loss curves
def plot_training_curves(history):

    # training and validation data accuracy
    acc     = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    # training and validation data loss
    loss     = history.history['loss']
    val_loss = history.history['val_loss']

    # plot accuracy
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()), 1])
    plt.title('Training and Validation Accuracy')

    # plot loss
    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0, 2.0])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

In [ ]:
training_step_size = 100000//TRAINING_NUM_EPOCHS
val_step_size = 10000//TRAINING_NUM_EPOCHS

# callbacks (learning rate schedule, model checkpointing during training)
callbacks = [tf.keras.callbacks.LearningRateScheduler(lr_schedule),
             tf.keras.callbacks.ModelCheckpoint(filepath=str(SAVE_MODEL_PATH), 
                                                save_best_only=True, 
                                                monitor='val_loss', 
                                                verbose=1)]

# training
initial_epoch_num = 0
history           = model.fit(x=train_ds_cachefile, 
                              epochs=TRAINING_NUM_EPOCHS, 
                              verbose=1, 
                              steps_per_epoch=training_step_size,
                              callbacks=callbacks, 
                              validation_data=valid_ds_cachefile,                               
                              validation_steps=val_step_size,
                              initial_epoch=initial_epoch_num)


# plot accuracy and loss curves
plot_training_curves(history)

In [ ]:
# test
test_loss, test_accuracy = model.evaluate(x=test_ds_nocache)
print('Test loss:     ', test_loss)
print('Test accuracy: ', test_accuracy)